In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import gaussian_kde
from pprint import pprint
from scipy.integrate import simpson
from collections import Counter
import logging
logging.basicConfig(level=logging.DEBUG)

from get_magic_numbers import get_magic_numbers_main

from string_magic_numbers import magic_strings_detection as string_values_process
from sign_violation_magic_numbers import sign_violation_magic_numbers as opposite_sign_process
from distance_based_magic_numbers import delta_distributed_magic_numbers 
from identical_magic_numbers import identical_column_magic_numbers as all_values_are_same
from magic_dictionaries import magic_dictionary, add_to_master_dict, safe_concatenate
from magic_dictionaries import clean_magic_results
from density_plot import plot_data_density


In [2]:
def synthetic_data(
        mean, 
        sigma, 
        num_samples=1000, 
        random_seed=None, 
        magic_values=[-999, 999],
        quantities=[100, 50], 
        col_names="synthetic_col"):

    if random_seed is not None:
        np.random.seed(random_seed)

    # Calculate Clipping Bounds (HARDCODED 3-SIGMA) ---
    # Hardcode sigma_limit = 3.0 to maintain the 99.7% for testing consistency
    SIGMA_LIMIT = 3.0  
    lower_bound = mean - SIGMA_LIMIT * sigma
    upper_bound = mean + SIGMA_LIMIT * sigma
    
    # Generate and Clip the Main Normal Distribution Samples 
    
    # Generate the initial samples
    synthetic_col = np.random.normal(mean, sigma, num_samples)
    
    # Apply 3-sigma clipping to ensure the normal data is within the 99.7% range
    synthetic_col = np.clip(synthetic_col, a_min=lower_bound, a_max=upper_bound)

    # --- 3. Add Magic Values (Original behavior: total size increases) ---

    if len(magic_values) != len(quantities):
        raise ValueError("Length of magic_values must match length of quantities")
    
    # If no magic values returns the normal distribution
    if len(magic_values) == 0:
        df = pd.DataFrame({col_names: synthetic_col})
        return df
    
    # Add magic values to the dataset
    for magic_value, quantity in zip(magic_values, quantities):
        magic_samples = np.full(quantity, magic_value)
        synthetic_col = np.concatenate((synthetic_col, magic_samples))
    
    # Shuffle the final array
    np.random.shuffle(synthetic_col)
    
    df = pd.DataFrame({col_names: synthetic_col})
    return df

# Note: If you use the parameters quantities=[100, 50] and num_samples=1000,
# the final dataset will have 1000 + 100 + 50 = 1150 samples.

In [3]:
def generate_randomized_df(n_columns, num_samples=1000):

    column_list = []
    
    for i in range(n_columns):
        random_mean = np.random.uniform(10, 500)    # Mean between 10 and 500
        random_sigma = np.random.uniform(1, 50)     # Sigma between 1 and 50
        
        df_col = synthetic_data(
            mean=random_mean, 
            sigma=random_sigma, 
            num_samples=num_samples, 
            col_names=i, # Sets the header to 0, 1,... n-1
            random_seed=np.random.randint(0, 100000) # Different seed for each column
        )
        column_list.append(df_col)
    
    # 3. Combine all columns into one DataFrame
    final_df = pd.concat(column_list, axis=1)
    
    return final_df

df_final = generate_randomized_df(n_columns=5, num_samples=1000)

# p is the value you want, k is naturally the number of rows in the df
p = 42 
df_final[len(df_final.columns)] = p

for times in range(10):
    random_row_index = np.random.randint(0, len(df_final))
    df_final.loc[random_row_index, 4] = "Flag"
print(df_final.describe()) 

                 0            1            2            3       5
count  1150.000000  1150.000000  1150.000000  1150.000000  1150.0
mean    375.345694   351.953106   -18.815977   384.158252    42.0
std     437.290883   434.123630   362.282264   440.075567     0.0
min    -999.000000  -999.000000  -999.000000  -999.000000    42.0
25%     473.123261   411.257790     5.646156   468.547298    42.0
50%     481.204856   451.920598    26.525713   490.429703    42.0
75%     488.360330   489.581515    46.408252   510.572018    42.0
max     999.000000   999.000000   999.000000   999.000000    42.0


C:\Users\mixal\AppData\Local\Temp\ipykernel_1856\3057800188.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Flag' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.loc[random_row_index, 4] = "Flag"


In [4]:
rows, columns = df_final.shape

extended_col_info =[]
for i in range(columns):
    column_info = []

    column_info.append(i)  # Column name/index
    column_info.append("N") # Indicate Numerical column
    column_info.append(rows)  # Number of rows
    extended_col_info.append(column_info)

In [5]:
magic_master_dict, magic_cleaned_dict = get_magic_numbers_main(df_final, extended_col_info, sign_violation_theshold = 3,gauss_threshold = 0.01, overlap_threshold=5.0, plot_graphs = False)

Processing column: 0 with 1150 entries.
  Found 0 string-based magic numbers.
  All values identical: False
  Analyzing numerical magic numbers in column: 0
  Found 1 sign-violation magic numbers.
  Found 2 distance-based magic numbers arrays.
  Total distance-based magic numbers found: 2
Processing column: 1 with 1150 entries.
  Found 0 string-based magic numbers.
  All values identical: False
  Analyzing numerical magic numbers in column: 1
  Found 1 sign-violation magic numbers.
  Found 2 distance-based magic numbers arrays.
  Total distance-based magic numbers found: 2
Processing column: 2 with 1150 entries.
  Found 0 string-based magic numbers.
  All values identical: False
  Analyzing numerical magic numbers in column: 2
  Found 0 sign-violation magic numbers.
  Found 2 distance-based magic numbers arrays.
  Total distance-based magic numbers found: 2
Processing column: 3 with 1150 entries.
  Found 0 string-based magic numbers.
  All values identical: False
  Analyzing numerical 

In [6]:
print ("Detected Magic Numbers:")
pprint(magic_master_dict)

Detected Magic Numbers:
{0: {'all_magic_numbers': False,
     'magic_distanced_numbers': array([ 999., -999.]),
     'magic_sign_violation': [-999.0],
     'magic_strings': []},
 1: {'all_magic_numbers': False,
     'magic_distanced_numbers': array([ 999., -999.]),
     'magic_sign_violation': [-999.0],
     'magic_strings': []},
 2: {'all_magic_numbers': False,
     'magic_distanced_numbers': array([ 999., -999.]),
     'magic_sign_violation': [],
     'magic_strings': []},
 3: {'all_magic_numbers': False,
     'magic_distanced_numbers': array([ 999., -999.]),
     'magic_sign_violation': [-999.0],
     'magic_strings': []},
 4: {'all_magic_numbers': False,
     'magic_distanced_numbers': array([ 999., -999.]),
     'magic_sign_violation': [],
     'magic_strings': ['Flag']},
 5: {'all_magic_numbers': 42,
     'magic_distanced_numbers': [],
     'magic_sign_violation': [],
     'magic_strings': []}}


In [7]:
pprint ("Cleaned Magic Numbers:")
pprint(magic_cleaned_dict)

'Cleaned Magic Numbers:'
{0: [-999.0, 999.0],
 1: [-999.0, 999.0],
 2: [-999.0, 999.0],
 3: [-999.0, 999.0],
 4: ['-999.0', '999.0', 'Flag']}


In [8]:
from astropy.io import ascii

readme_url = "https://cdsarc.cds.unistra.fr/ftp/J/A+A/469/861/ReadMe"
data_url = "https://cdsarc.cds.unistra.fr/ftp/J/A+A/469/861/table1.dat"

table = ascii.read(data_url, readme=readme_url, format="cds")
df = table.to_pandas()
print(df[:9])


   [BBG2007b] n_[BBG2007b]              Name  RAh  RAm    RAs DE-  DEd  DEm  \
0           1          NaN  J005535.4+261752    0   55  35.44   +   26   17   
1           2          NaN  J005536.0+262711    0   55  36.06   +   26   27   
2           3          NaN  J005539.1+261457    0   55  39.16   +   26   14   
3           4          NaN  J005539.4+261531    0   55  39.45   +   26   15   
4           5          NaN  J005540.1+262056    0   55  40.13   +   26   20   
5           6          NaN  J005544.5+261318    0   55  44.52   +   26   13   
6           7          NaN  J005544.5+262755    0   55  44.54   +   26   27   
7           8            F  J005544.5+262819    0   55  44.56   +   26   28   
8           9          NaN  J005544.8+262627    0   55  44.86   +   26   26   

    DEs   Bmag   Rmag     RV  e_RV SpClass  
0  52.8  21.38  19.15  58643   135     NaN  
1  11.7  20.39  18.62  57857   157     NaN  
2  57.2  20.67  18.75  59498    90       k  
3  31.8  21.99  19.91  58948 

In [9]:
df = table.to_pandas()

rows, columns = df.shape

extended_col_info =[]
for i in range(columns):
    column_info = []

    column_header = df.columns[i] # Get the header of each column
    column_info.append(column_header) 

    column_type = df[column_header].dtype
    if pd.api.types.is_float_dtype(column_type) == True or pd.api.types.is_integer_dtype(column_type) == True:
        column_info.append("N")  # Numeric column
    elif pd.api.types.is_string_dtype(column_type) == True:
        column_info.append("S")  # String column
        
    column_info.append(i)  # Column name/index
    column_info.append(rows)  # Number of rows
    extended_col_info.append(column_info)

pprint(extended_col_info)

[['[BBG2007b]', 'N', 0, 115],
 ['n_[BBG2007b]', 'S', 1, 115],
 ['Name', 'S', 2, 115],
 ['RAh', 'N', 3, 115],
 ['RAm', 'N', 4, 115],
 ['RAs', 'N', 5, 115],
 ['DE-', 'S', 6, 115],
 ['DEd', 'N', 7, 115],
 ['DEm', 'N', 8, 115],
 ['DEs', 'N', 9, 115],
 ['Bmag', 'N', 10, 115],
 ['Rmag', 'N', 11, 115],
 ['RV', 'N', 12, 115],
 ['e_RV', 'N', 13, 115],
 ['SpClass', 'S', 14, 115]]


In [10]:
magic_master_dict, magic_cleaned_dict = get_magic_numbers_main(df, extended_col_info, sign_violation_theshold = 3,gauss_threshold = 0.01, overlap_threshold=5.0, plot_graphs = False)

Processing column: [BBG2007b] with 115 entries.
  Found 0 string-based magic numbers.
  All values identical: False
  Analyzing numerical magic numbers in column: [BBG2007b]
  Found 0 sign-violation magic numbers.
  Found 0 distance-based magic numbers arrays.
  Total distance-based magic numbers found: 0
Processing column: RAh with 115 entries.
  Found 0 string-based magic numbers.
  All values identical: 0
  Skipping further analysis for column RAh as all values are identical.
Processing column: RAm with 115 entries.
  Found 0 string-based magic numbers.
  All values identical: False
  Analyzing numerical magic numbers in column: RAm
  Found 0 sign-violation magic numbers.
  Found 0 distance-based magic numbers arrays.
  Total distance-based magic numbers found: 0
Processing column: RAs with 115 entries.
  Found 0 string-based magic numbers.
  All values identical: False
  Analyzing numerical magic numbers in column: RAs
  Found 0 sign-violation magic numbers.
  Found 0 distance-base

In [11]:
print ("Detected Magic Numbers:")
pprint(magic_master_dict)

Detected Magic Numbers:
{'Bmag': {'all_magic_numbers': False,
          'magic_distanced_numbers': array([], dtype=float64),
          'magic_sign_violation': [],
          'magic_strings': []},
 'DE-': {'all_magic_numbers': None,
         'magic_distanced_numbers': [],
         'magic_sign_violation': [],
         'magic_strings': []},
 'DEd': {'all_magic_numbers': 26,
         'magic_distanced_numbers': [],
         'magic_sign_violation': [],
         'magic_strings': []},
 'DEm': {'all_magic_numbers': False,
         'magic_distanced_numbers': array([], dtype=float64),
         'magic_sign_violation': [],
         'magic_strings': []},
 'DEs': {'all_magic_numbers': False,
         'magic_distanced_numbers': array([0.2]),
         'magic_sign_violation': [],
         'magic_strings': []},
 'Name': {'all_magic_numbers': None,
          'magic_distanced_numbers': [],
          'magic_sign_violation': [],
          'magic_strings': []},
 'RAh': {'all_magic_numbers': 0,
         'magic_

In [12]:
pprint ("Cleaned Magic Numbers:")
pprint(magic_cleaned_dict)
# Note: that the non numeric columns are ignored in the magic number detection 
# and in the cleaned results are last with None indicating it.


'Cleaned Magic Numbers:'
{'Bmag': [],
 'DE-': None,
 'DEd': 26,
 'DEm': [],
 'DEs': [0.2],
 'Name': None,
 'RAh': [],
 'RAm': [],
 'RAs': [],
 'RV': [],
 'Rmag': [],
 'SpClass': None,
 '[BBG2007b]': [],
 'e_RV': [],
 'n_[BBG2007b]': None}
